In [ ]:
!pip install tensorflow
!pip install scikit-learn
!pip install torch torchvision
!pip install torch-geometric
!pip install optuna
!pip install scikit-learn
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install timm
# # or for huggingface transformers if you'd like to use that:
!pip install transformers
!pip install scikit-learn
!pip install matplotlib opencv-python
!pip install tensorflow
!pip install keras-tuner

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sivm205/soybean-diseased-leaf-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/soybean-diseased-leaf-dataset


In [3]:
import os
import random
import numpy as np
from PIL import Image

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models

# PyTorch Geometric imports for the GNN branch
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Parallel CNN+GNN Model for Image Classification

This script implements a parallel architecture in which an image is processed
simultaneously by a CNN branch (using MobileNetV2) and a GNN branch (using a GCN).
The CNN branch extracts spatial features from the image, while the GNN branch converts
the image into a graph (each patch becomes a node with the mean RGB value) and processes
the graph structure. The resulting features are fused and passed to a classification head.

Author: [Your Name]
Date: [Current Date]
"""

# -----------------------------
# 1. Set Random Seeds for Reproducibility
# -----------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# -----------------------------
# 2. Image-to-Graph Conversion Function
# -----------------------------
def image_to_graph(image: np.ndarray, patch_size=(16, 16)) -> Data:
    """
    Converts an image (H, W, C) into a graph (PyG Data object) by splitting it into patches.
    
    Each patch is represented by its mean RGB value (node feature). Nodes are connected
    to their immediate 4-neighbors (up, down, left, right).

    Args:
        image (np.ndarray): Input image with shape [H, W, 3].
        patch_size (tuple): Size (height, width) of each patch.

    Returns:
        Data: A PyTorch Geometric Data object with node features 'x' and edge_index.
    """
    h, w, c = image.shape
    ph, pw = patch_size
    num_nodes_h = h // ph
    num_nodes_w = w // pw
    nodes = []

    # Compute mean RGB for each patch
    for i in range(num_nodes_h):
        for j in range(num_nodes_w):
            patch = image[i*ph:(i+1)*ph, j*pw:(j+1)*pw, :]
            patch_feature = patch.mean(axis=(0, 1))
            nodes.append(patch_feature)
    nodes = np.array(nodes, dtype=np.float32)

    # Build edge_index for 4-connected grid
    edge_index = []
    for i in range(num_nodes_h):
        for j in range(num_nodes_w):
            node_idx = i * num_nodes_w + j
            # Up neighbor
            if i > 0:
                neighbor = (i - 1) * num_nodes_w + j
                edge_index.append([node_idx, neighbor])
                edge_index.append([neighbor, node_idx])
            # Down neighbor
            if i < num_nodes_h - 1:
                neighbor = (i + 1) * num_nodes_w + j
                edge_index.append([node_idx, neighbor])
                edge_index.append([neighbor, node_idx])
            # Left neighbor
            if j > 0:
                neighbor = i * num_nodes_w + (j - 1)
                edge_index.append([node_idx, neighbor])
                edge_index.append([neighbor, node_idx])
            # Right neighbor
            if j < num_nodes_w - 1:
                neighbor = i * num_nodes_w + (j + 1)
                edge_index.append([node_idx, neighbor])
                edge_index.append([neighbor, node_idx])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Create the Data object
    x = torch.tensor(nodes, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index)
    return data

# -----------------------------
# 3. Custom Dataset for Parallel CNN+GNN
# -----------------------------
class ParallelCnnGnnDataset(Dataset):
    """
    Dataset that returns a tuple for each image:
      - A CNN-processed image tensor.
      - A graph (PyG Data object) constructed from the image.
      - The class label.
      
    The dataset assumes a folder structure:
        root_dir/
            class_0/
                img1.jpg, img2.jpg, ...
            class_1/
                ...
    """
    def __init__(self, root_dir, cnn_transform=None, graph_patch_size=(16, 16), image_size=(224, 224)):
        """
        Args:
            root_dir (str): Path to the dataset directory.
            cnn_transform: Transformations for the CNN branch (e.g., Resize, ToTensor, Normalize).
            graph_patch_size (tuple): Patch size for graph conversion.
            image_size (tuple): Final image size for CNN input.
        """
        self.root_dir = root_dir
        self.cnn_transform = cnn_transform
        self.graph_patch_size = graph_patch_size
        self.image_size = image_size
        
        # List all subdirectories as classes (sorted for consistency)
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        # Build list of (image_path, label) tuples
        self.samples = []
        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            for fname in os.listdir(cls_path):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.samples.append((os.path.join(cls_path, fname), self.class_to_idx[cls]))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        # Open image and ensure RGB
        with Image.open(path) as img:
            img = img.convert('RGB')
            # For CNN branch: apply transformation if provided
            if self.cnn_transform:
                cnn_img = self.cnn_transform(img)
            else:
                img_resized = img.resize(self.image_size)
                cnn_img = transforms.ToTensor()(img_resized)
            
            # For GNN branch: resize (if needed) and convert to numpy array
            img_for_graph = img.resize(self.image_size)
            img_np = np.array(img_for_graph)
        
        # Convert image to graph
        graph_data = image_to_graph(img_np, patch_size=self.graph_patch_size)
        graph_data.y = torch.tensor([label], dtype=torch.long)
        
        return cnn_img, graph_data, label

# -----------------------------
# 4. Custom Collate Function for DataLoader
# -----------------------------
def parallel_collate_fn(batch):
    """
    Collate function to merge a list of samples into a batch.
    
    Each sample is a tuple (cnn_img, graph_data, label). This function stacks the CNN images
    into a tensor and batches the graph objects using PyG's Batch.from_data_list.
    """
    cnn_imgs, graph_data_list, labels = [], [], []
    for cnn_img, g_data, label in batch:
        cnn_imgs.append(cnn_img)
        graph_data_list.append(g_data)
        labels.append(label)
    
    cnn_batch = torch.stack(cnn_imgs, dim=0)
    gnn_batch = Batch.from_data_list(graph_data_list)
    labels = torch.tensor(labels, dtype=torch.long)
    return cnn_batch, gnn_batch, labels

# -----------------------------
# 5. Parallel CNN+GNN Model Definition
# -----------------------------
class ParallelCNNGNN(nn.Module):
    """
    A parallel model that fuses features from a CNN branch (MobileNetV2) and a GNN branch.
    
    The CNN branch extracts features from the image, while the GNN branch extracts graph-level
    features from a patch-based representation of the same image. The features are concatenated
    and fed to a final classifier.
    """
    def __init__(self, num_classes, gnn_num_layers=2, gnn_hidden_dim=64, gnn_dropout=0.5, freeze_cnn=False):
        """
        Args:
            num_classes (int): Number of classes.
            gnn_num_layers (int): Number of GCNConv layers.
            gnn_hidden_dim (int): Hidden dimension for GNN layers.
            gnn_dropout (float): Dropout probability for the GNN branch.
            freeze_cnn (bool): If True, freezes the CNN branch.
        """
        super(ParallelCNNGNN, self).__init__()
        
        # --- CNN Branch: MobileNetV2 ---
        # Load pretrained MobileNetV2 (with weights on ImageNet)
        self.cnn = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        # Remove the classifier head; we use only the feature extractor.
        self.cnn.classifier = nn.Identity()
        if freeze_cnn:
            for param in self.cnn.parameters():
                param.requires_grad = False
        
        # Global pooling to reduce CNN feature maps to a vector.
        self.global_pool_cnn = nn.AdaptiveAvgPool2d((1, 1))
        
        # --- GNN Branch ---
        # For image-to-graph conversion, the node features are of size 3 (mean RGB).
        in_feats = 3
        self.convs = nn.ModuleList()
        self.convs.append(GCNConv(in_feats, gnn_hidden_dim))
        for _ in range(gnn_num_layers - 1):
            self.convs.append(GCNConv(gnn_hidden_dim, gnn_hidden_dim))
        self.gnn_dropout = gnn_dropout
        
        # --- Fusion and Final Classification ---
        # CNN branch outputs 1280 features (for MobileNetV2) and GNN branch outputs gnn_hidden_dim.
        self.fc_fusion = nn.Linear(1280 + gnn_hidden_dim, num_classes)
    
    def forward(self, cnn_batch, gnn_batch):
        # ----- CNN Branch -----
        # Extract features using the MobileNetV2 feature extractor.
        x_cnn = self.cnn.features(cnn_batch)          # [B, 1280, H_feat, W_feat]
        x_cnn = self.global_pool_cnn(x_cnn)             # [B, 1280, 1, 1]
        x_cnn = x_cnn.view(x_cnn.size(0), -1)           # [B, 1280]
        
        # ----- GNN Branch -----
        x_gnn = gnn_batch.x                           # Node features: [total_nodes, 3]
        edge_index = gnn_batch.edge_index             # Edge indices
        batch_idx = gnn_batch.batch                   # Batch vector mapping nodes to graphs
        
        # Pass through each GCN layer with ReLU activation and dropout.
        for conv in self.convs:
            x_gnn = conv(x_gnn, edge_index)
            x_gnn = F.relu(x_gnn)
            x_gnn = F.dropout(x_gnn, p=self.gnn_dropout, training=self.training)
        
        # Global pooling to obtain graph-level features.
        x_gnn = global_mean_pool(x_gnn, batch_idx)      # [B, gnn_hidden_dim]
        
        # ----- Fusion -----
        x_fused = torch.cat([x_cnn, x_gnn], dim=1)       # [B, 1280 + gnn_hidden_dim]
        logits = self.fc_fusion(x_fused)                # [B, num_classes]
        return logits

# -----------------------------
# 6. Training and Evaluation Routine
# -----------------------------
def train_parallel_cnn_gnn():
    # ----------- Configuration -----------
    data_dir = "/kaggle/input/soybean-diseased-leaf-dataset"  # Update path as needed
    batch_size = 8
    num_epochs = 10
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Determine the number of classes based on subdirectories in the dataset
    classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    num_classes = len(classes)
    print(f"Detected {num_classes} classes: {classes}")
    
    # ----------- Transforms for CNN Branch -----------
    cnn_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    
    # ----------- Create Dataset and DataLoaders -----------
    dataset = ParallelCnnGnnDataset(root_dir=data_dir,
                                    cnn_transform=cnn_transform,
                                    graph_patch_size=(16, 16),
                                    image_size=(224, 224))
    # Split dataset: 80% training, 20% validation
    total_samples = len(dataset)
    indices = list(range(total_samples))
    split = int(0.2 * total_samples)
    val_indices = indices[:split]
    train_indices = indices[split:]
    
    train_dataset = Subset(dataset, train_indices)
    val_dataset   = Subset(dataset, val_indices)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              collate_fn=parallel_collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                            collate_fn=parallel_collate_fn)
    
    # ----------- Initialize Model, Loss, and Optimizer -----------
    model = ParallelCNNGNN(num_classes=num_classes,
                           gnn_num_layers=2,
                           gnn_hidden_dim=64,
                           gnn_dropout=0.5,
                           freeze_cnn=False)
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    
    # ----------- Training Loop -----------
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for cnn_batch, gnn_batch, labels in train_loader:
            cnn_batch = cnn_batch.to(device)
            gnn_batch = gnn_batch.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(cnn_batch, gnn_batch)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * cnn_batch.size(0)
        
        avg_loss = total_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}] Training Loss: {avg_loss:.4f}")
        
        # ----------- Validation -----------
        model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for cnn_batch, gnn_batch, labels in val_loader:
                cnn_batch = cnn_batch.to(device)
                gnn_batch = gnn_batch.to(device)
                labels = labels.to(device)
                outputs = model(cnn_batch, gnn_batch)
                _, preds = torch.max(outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        print(f"Epoch [{epoch+1}/{num_epochs}] Validation Accuracy: {val_acc:.4f}")
    
    # ----------- Final Evaluation on Validation Set -----------
    prec = precision_score(val_labels, val_preds, average='weighted', zero_division=0)
    rec  = recall_score(val_labels, val_preds, average='weighted', zero_division=0)
    f1   = f1_score(val_labels, val_preds, average='weighted', zero_division=0)
    
    print("\nFinal Evaluation Metrics on Validation Set:")
    print(f"Accuracy:  {val_acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}\n")
    print("Classification Report:")
    print(classification_report(val_labels, val_preds, zero_division=0))

# -----------------------------
# 7. Main Execution
# -----------------------------
if __name__ == '__main__':
    train_parallel_cnn_gnn()

Detected 10 classes: ['Mossaic Virus', 'Southern blight', 'Sudden Death Syndrone', 'Yellow Mosaic', 'bacterial_blight', 'brown_spot', 'crestamento', 'ferrugen', 'powdery_mildew', 'septoria']


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 110MB/s] 


Epoch [1/10] Training Loss: 0.9164
Epoch [1/10] Validation Accuracy: 0.3058
Epoch [2/10] Training Loss: 0.1864
Epoch [2/10] Validation Accuracy: 0.3058
Epoch [3/10] Training Loss: 0.1286
Epoch [3/10] Validation Accuracy: 0.3058
Epoch [4/10] Training Loss: 0.0501
Epoch [4/10] Validation Accuracy: 0.3058
Epoch [5/10] Training Loss: 0.0525
Epoch [5/10] Validation Accuracy: 0.3058
Epoch [6/10] Training Loss: 0.0343
Epoch [6/10] Validation Accuracy: 0.3058
Epoch [7/10] Training Loss: 0.0595
Epoch [7/10] Validation Accuracy: 0.3058
Epoch [8/10] Training Loss: 0.0603
Epoch [8/10] Validation Accuracy: 0.3058
Epoch [9/10] Training Loss: 0.0146
Epoch [9/10] Validation Accuracy: 0.3058
Epoch [10/10] Training Loss: 0.0165
Epoch [10/10] Validation Accuracy: 0.3058

Final Evaluation Metrics on Validation Set:
Accuracy:  0.3058
Precision: 0.3058
Recall:    0.3058
F1 Score:  0.3058

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.0

# Working with New Parallel CNN and GNN

In [ ]:
!pip install tensorflow numpy networkx scipy opencv-python scikit-learn optuna

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import networkx as nx
import scipy.sparse as sp
import os
import cv2
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import optuna
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [9]:
# Dataset Directory
dataset_dir = "/kaggle/input/soybean-diseased-leaf-dataset"

# Load dataset images and labels
def load_dataset(dataset_dir, img_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(dataset_dir))
    class_dict = {class_name: idx for idx, class_name in enumerate(class_names)}
    
    for class_name in class_names:
        class_path = os.path.join(dataset_dir, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            img = img / 255.0  # Normalize image
            images.append(img)
            labels.append(class_dict[class_name])
    
    return np.array(images), np.array(labels)

# Load actual dataset
images, labels = load_dataset(dataset_dir)

# Data Preprocessing & Augmentation
def preprocess_data(X, y):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        rescale=1./255
    )
    return datagen.flow(X, y, batch_size=32)

# Load MobileNetV2 as feature extractor
def create_cnn(input_shape):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    model = tf.keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu')
    ])
    return model

# Extract features from images using MobileNetV2
def extract_features(images):
    cnn_model = create_cnn((224, 224, 3))
    return cnn_model.predict(images)

# Construct graph using cosine similarity of image features
def create_graph(image_features, threshold=0.8):
    similarity_matrix = cosine_similarity(image_features)
    adj_matrix = (similarity_matrix > threshold).astype(int)
    return sp.coo_matrix(adj_matrix)

# Graph Convolutional Network (GCN) Layer
def create_gcn(adj_matrix, features):
    input_features = tf.keras.Input(shape=(features.shape[1],))
    x = layers.Dense(64, activation='relu')(input_features)
    x = layers.Dense(32, activation='relu')(x)
    output = layers.Dense(128, activation='relu')(x)
    return tf.keras.Model(inputs=input_features, outputs=output)

# Hybrid Model Combining CNN & GCN
def create_hybrid_model(input_shape, adj_matrix, node_features):
    cnn_model = create_cnn(input_shape)
    gcn_model = create_gcn(adj_matrix, node_features)
    
    cnn_input = tf.keras.Input(shape=input_shape)
    gcn_input = tf.keras.Input(shape=(node_features.shape[1],))
    
    cnn_output = cnn_model(cnn_input)
    gcn_output = gcn_model(gcn_input)
    
    merged = layers.Concatenate()([cnn_output, gcn_output])
    final_output = layers.Dense(10, activation='softmax')(merged)
    
    model = tf.keras.Model(inputs=[cnn_input, gcn_input], outputs=final_output)
    return model

# Evaluate model performance
def evaluate_model(model, test_data, test_labels):
    test_labels = np.argmax(test_labels, axis=1)  # Convert one-hot encoding to categorical labels
    
    predictions = model.predict(test_data)
    predicted_labels = np.argmax(predictions, axis=1)

    acc = accuracy_score(test_labels, predicted_labels)
    precision = precision_score(test_labels, predicted_labels, average='weighted')
    recall = recall_score(test_labels, predicted_labels, average='weighted')
    f1 = f1_score(test_labels, predicted_labels, average='weighted')

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return acc, precision, recall, f1

# Define objective function for Optuna
def objective(trial):
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    
    model = create_hybrid_model((224, 224, 3), adj_matrix, node_features)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit([train_data, node_features], tf.keras.utils.to_categorical(train_labels, num_classes=10), epochs=5, batch_size=batch_size, validation_split=0.2, verbose=0)
    return max(history.history['val_accuracy'])

# Extract features and create graph
node_features = extract_features(images)
adj_matrix = create_graph(node_features)

# Train-Test Split
train_data, test_data, train_labels, test_labels, train_node_features, test_node_features = train_test_split(images, labels, node_features, test_size=0.2, random_state=42)

# Run Hyperparameter Optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Train Final Model with Best Parameters
best_params = study.best_params
final_model = create_hybrid_model((224, 224, 3), adj_matrix, node_features)
final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['lr']), loss='categorical_crossentropy', metrics=['accuracy'])
final_model.fit([train_data, train_node_features], tf.keras.utils.to_categorical(train_labels, num_classes=10), epochs=20, batch_size=best_params['batch_size'], validation_data=([test_data, test_node_features], tf.keras.utils.to_categorical(test_labels, num_classes=10)))

# Evaluate and Save Model
evaluate_model(final_model, [test_data, test_node_features], tf.keras.utils.to_categorical(test_labels, num_classes=10))
final_model.save('hybrid_model.h5')

22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step


[I 2025-02-10 17:26:36,153] A new study created in memory with name: no-name-a83d09e1-6ee7-48cb-a00e-de2f9eddf190
[I 2025-02-10 17:26:50,910] Trial 0 finished with value: 0.4732142984867096 and parameters: {'lr': 3.3670945851963566e-05, 'batch_size': 64}. Best is trial 0 with value: 0.4732142984867096.
[I 2025-02-10 17:27:07,994] Trial 1 finished with value: 0.2410714328289032 and parameters: {'lr': 1.2513966301630107e-05, 'batch_size': 64}. Best is trial 0 with value: 0.4732142984867096.
[I 2025-02-10 17:27:22,994] Trial 2 finished with value: 0.4375 and parameters: {'lr': 2.2128817338785996e-05, 'batch_size': 64}. Best is trial 0 with value: 0.4732142984867096.
[I 2025-02-10 17:27:38,338] Trial 3 finished with value: 0.8035714030265808 and parameters: {'lr': 0.00011781646662243641, 'batch_size': 64}. Best is trial 3 with value: 0.8035714030265808.
[I 2025-02-10 17:27:53,550] Trial 4 finished with value: 0.4732142984867096 and parameters: {'lr': 4.001247355729215e-05, 'batch_size': 64

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 404ms/step - accuracy: 0.6504 - loss: 1.1572 - val_accuracy: 0.9504 - val_loss: 0.2260
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.9746 - loss: 0.1134 - val_accuracy: 0.9291 - val_loss: 0.1943
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9892 - loss: 0.0642 - val_accuracy: 0.9078 - val_loss: 0.2660
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9723 - loss: 0.0938 - val_accuracy: 0.9504 - val_loss: 0.2291
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9923 - loss: 0.0258 - val_accuracy: 0.9574 - val_loss: 0.1383
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9977 - loss: 0.0143 - val_accuracy: 0.9645 - val_loss: 0.1309
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9935 - loss: 0.0208 - val_accuracy: 0.9504 - val_loss: 0.1266
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9942 - loss: 0.0105 - val_accuracy: 0.9645 -